#**ArtRecSys Challenge**

Public leaderboard score (NDCG@20): 0.14713

# Загрузка данных из Kaggle

In [1]:
!pip install kaggle --upgrade

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)


In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "tselischevivaniad5" # username from the json file
os.environ['KAGGLE_KEY'] = "17a8310682739ae6a90aeefaf8e97d48" # key from the json file

In [3]:
!kaggle competitions download -c recsys-iad-challenge

  0% 0.00/10.9M [00:00<?, ?B/s]
100% 10.9M/10.9M [00:00<00:00, 101MB/s]
 99% 363M/368M [00:02<00:00, 160MB/s]
100% 368M/368M [00:02<00:00, 150MB/s]
 90% 212M/235M [00:01<00:00, 184MB/s]
100% 235M/235M [00:01<00:00, 202MB/s]


In [4]:
!unzip \*.zip

Archive:  train.json.zip
  inflating: train.json              

Archive:  items.json.zip
  inflating: items.json              

Archive:  random_benchmark.csv.zip
  inflating: random_benchmark.csv    

3 archives were successfully processed.


# Матрицы для модели

In [0]:
import pandas as pd
import json

In [6]:
!pip install lightfm

     |████████████████████████████████| 307kB 2.8MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707618 sha256=4fd3347e0791b4ca361ec36db2ac398e47cbc4d298f36fe20afcbf5a062d52b8
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [0]:
from scipy.sparse import coo_matrix, hstack, csr_matrix
import gc
from lightfm import LightFM
from tqdm.auto import tqdm

**User-Item матрица**

In [0]:
def user_item():
  row  = []
  col  = []
  data = []
  with tqdm(open('train.json')) as f:
      for i, line in enumerate(f):
          n = json.loads(line)
          for j, (key, value) in enumerate(n['trainRatings'].items()):
              row.append(n['userId'])
              col.append(int(key))
              if value == 0:
                data.append(value-1)
              else:
                data.append(value)

  coo = coo_matrix((data, (row, col)))
  print('User-item matrix created, shape : '+ str(coo.get_shape()))
  del [row, col, data]
  gc.collect()
  return coo

In [9]:
ui_matrix = user_item()


User-item matrix created, shape : (42977, 328050)


**Таблица items, признаки**

In [0]:
import numpy as np

In [0]:
with open('items.json') as f:
    items = pd.DataFrame(json.loads(line) for line in f)

In [12]:
print(items.shape)
items.head()

(328050, 4)


,content,itemId,image,title
0,"Согласитесь, дорогие любители собак, до чего ж...",0,"[-0.169, 0.129, 0.067, 0.019, 0.281, -0.245, 0...",Пять забавных «морщинистых» пород собак
1,"Контуры Третьей Поперечной улицы, состоявшей ...",1,"[-0.158, -0.112, -0.325, 0.05, -0.114, 0.002, ...",История улицы Ирининской в Гомеле
2,Источник: http://infodays.ru Вообще он как-то ...,2,"[0.084, -0.181, 0.008, 0.34, -0.03, -0.197, -0...",Зачем Дудь всё время спрашивает гостей програм...
3,41-летняя Светлана Зейналова решила окрестить ...,3,"[0.034, -0.119, -0.062, 0.025, 0.128, -0.041, ...",Светлана Зейналова крестила младшую дочь
4,«Организованные преступные группировки ГБАО де...,4,"[-0.061, -0.015, -0.198, -0.047, 0.054, 0.029,...","ГКНБ: бандиты в ГБАО делают вид, что рассталис..."


In [13]:
items.isnull().sum()

content       0
itemId        0
image      7226
title         0
dtype: int64

In [0]:
l = list()
s = set()
for i in items.image:
  if type(i) is float:
    l.append(i)
  else:
    s.add(len(i))

In [15]:
s

{96}

Размер эмбеддингов = 96, есть пустые-nan

In [0]:
def func(x):
    if type(x) is float:
      x = [0]*96
    return x

Заполним пустые эмбеддинги нулями

In [0]:
items.image = items.image.apply(lambda x: func(x))

Первыми признаками будут матрицы полученные из TFIDF для контента и заголовков

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vectorizer = TfidfVectorizer(lowercase=False, max_df=0.01, min_df=10)
matrix_tf = vectorizer.fit_transform(items.title.to_list())

In [20]:
matrix_tf

<328050x29762 sparse matrix of type '<class 'numpy.float64'>'
	with 1641340 stored elements in Compressed Sparse Row format>

In [21]:
%%time
vectorizer2 = TfidfVectorizer(lowercase=False, max_df=0.01, min_df=10)
matrix_tf2 = vectorizer2.fit_transform(items.content.to_list())

CPU times: user 1min 16s, sys: 812 ms, total: 1min 17s
Wall time: 1min 17s


In [22]:
matrix_tf2

<328050x239705 sparse matrix of type '<class 'numpy.float64'>'
	with 27815164 stored elements in Compressed Sparse Row format>

In [23]:
items.isnull().sum()

content    0
itemId     0
image      0
title      0
dtype: int64

Сделаем из столбца эмбеддингов 96 признаков

In [0]:
for i in range(96):
  items['emb'+str(i)] = items.image.apply(lambda x: x[i])

Добавим признаки : длина контента и длина заголовка, отмасштабируем их

In [0]:
items['title_length'] = items.title.apply(lambda x: len(x))
items['content_length'] = items.content.apply(lambda x: len(x))

In [0]:
items[['title_length', 'content_length']] = (items[['title_length', 'content_length']] - items[['title_length', 'content_length']].min()) / (items[['title_length', 'content_length']].max() - items[['title_length', 'content_length']].min())

Также добавим еще два признака : процент стоп-слов в контенте и заголовке

In [27]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [28]:
%%time
from nltk.corpus import stopwords
stop = stopwords.words('russian')
items['bad_content_percent'] = items.content.apply(lambda x: len([item for item in x if item in stop])/len(x) if len(x) != 0 else 0)
items['bad_title_percent'] = items.title.apply(lambda x: len([item for item in x if item in stop])/len(x) if len(x) != 0 else 0)

CPU times: user 9min 50s, sys: 333 ms, total: 9min 51s
Wall time: 9min 51s


In [29]:
items.head(3)

,content,itemId,image,title,emb0,emb1,emb2,emb3,emb4,emb5,emb6,emb7,emb8,emb9,emb10,emb11,emb12,emb13,emb14,emb15,emb16,emb17,emb18,emb19,emb20,emb21,emb22,emb23,emb24,emb25,emb26,emb27,emb28,emb29,emb30,emb31,emb32,emb33,emb34,emb35,...,emb60,emb61,emb62,emb63,emb64,emb65,emb66,emb67,emb68,emb69,emb70,emb71,emb72,emb73,emb74,emb75,emb76,emb77,emb78,emb79,emb80,emb81,emb82,emb83,emb84,emb85,emb86,emb87,emb88,emb89,emb90,emb91,emb92,emb93,emb94,emb95,title_length,content_length,bad_content_percent,bad_title_percent
0,"Согласитесь, дорогие любители собак, до чего ж...",0,"[-0.169, 0.129, 0.067, 0.019, 0.281, -0.245, 0...",Пять забавных «морщинистых» пород собак,-0.169,0.129,0.067,0.019,0.281,-0.245,0.089,0.329,0.072,-0.334,0.623,-0.029,0.126,-0.119,-0.075,0.175,-0.065,0.058,-0.249,-0.079,-0.057,0.170,-0.136,-0.031,0.039,0.058,0.103,0.068,-0.274,-0.068,-0.205,-0.077,0.256,-0.046,-0.027,-0.192,...,-0.078,0.292,0.256,-0.131,-0.055,0.040,0.330,0.330,0.146,0.332,0.139,0.178,-0.088,0.075,0.147,-0.235,-0.125,0.321,0.118,0.258,0.028,-0.397,0.008,-0.110,0.194,0.352,0.115,0.110,0.219,0.064,-0.054,-0.030,0.056,-0.428,-0.128,0.145,0.383838,1.0000,0.382000,0.358974
1,"Контуры Третьей Поперечной улицы, состоявшей ...",1,"[-0.158, -0.112, -0.325, 0.05, -0.114, 0.002, ...",История улицы Ирининской в Гомеле,-0.158,-0.112,-0.325,0.050,-0.114,0.002,-0.037,-0.373,0.056,-0.088,0.015,0.221,0.115,-0.070,0.309,0.255,0.030,0.040,0.190,0.117,0.126,-0.049,0.043,0.142,0.130,-0.241,0.088,0.082,0.003,-0.135,-0.020,-0.156,-0.138,0.110,0.036,0.072,...,-0.004,0.171,-0.101,0.132,-0.091,-0.056,-0.013,-0.366,-0.027,0.085,0.111,0.295,-0.080,0.008,-0.023,0.073,-0.097,-0.215,-0.369,-0.036,0.078,-0.171,-0.077,-0.010,-0.158,0.204,-0.198,-0.180,-0.055,-0.055,-0.132,0.084,-0.188,0.277,0.042,0.149,0.333333,1.0000,0.355500,0.382353
2,Источник: http://infodays.ru Вообще он как-то ...,2,"[0.084, -0.181, 0.008, 0.34, -0.03, -0.197, -0...",Зачем Дудь всё время спрашивает гостей програм...,0.084,-0.181,0.008,0.340,-0.030,-0.197,-0.051,-0.016,-0.024,-0.310,0.041,-0.129,-0.189,-0.100,-0.121,0.013,0.149,0.021,0.025,-0.231,-0.118,0.268,-0.120,0.001,-0.021,0.162,0.002,-0.139,-0.013,0.166,0.000,-0.038,-0.083,0.112,-0.089,0.128,...,-0.143,0.105,0.044,-0.048,0.032,-0.097,-0.132,-0.058,-0.108,-0.038,0.046,0.168,0.030,-0.055,0.002,-0.084,0.212,0.265,-0.132,-0.142,0.017,-0.118,-0.026,-0.286,0.164,-0.094,-0.284,0.012,-0.148,0.111,-0.043,0.123,0.093,-0.108,-0.359,-0.127,0.555556,0.5015,0.370887,0.339286


**Item-Feature матрица**

In [0]:
def item_feature():
  coo = csr_matrix(items.loc[:,'emb0':])
  coo = hstack([coo, matrix_tf, matrix_tf2])
  print('Item-feature matrix created, shape : '+ str(coo.get_shape()))
  return coo

In [31]:
if_matrix = item_feature()

Item-feature matrix created, shape : (328050, 269567)


In [32]:
del items
gc.collect()

0

# Обучение, сохранение

**Входные данные для модели**

In [33]:
ui_matrix

<42977x328050 sparse matrix of type '<class 'numpy.int64'>'
	with 67780168 stored elements in COOrdinate format>

In [34]:
if_matrix

<328050x269567 sparse matrix of type '<class 'numpy.float64'>'
	with 61485751 stored elements in COOrdinate format>

**Сохранение матриц**

In [0]:
from scipy.sparse import save_npz
save_npz("inters.npz", ui_matrix)
save_npz("ifeat.npz", if_matrix)

**Обучение**

In [0]:
%%time
import pickle

model = LightFM(no_components=60, learning_schedule='adagrad', loss='logistic', learning_rate=0.05, random_state=13)
for i in range(5):
  print('Epoch'+str(i))
  model.fit_partial(ui_matrix, item_features=if_matrix, epochs=1, num_threads=4)
  with open(str(i)+'model.pickle', 'wb') as fle:
    pickle.dump(model, fle, protocol=pickle.HIGHEST_PROTOCOL)

Epoch0
Epoch1
Epoch2
Epoch3
Epoch4
CPU times: user 1d 10h 16min 23s, sys: 21.3 s, total: 1d 10h 16min 44s
Wall time: 8h 38min 15s


**Загрузка модели и матриц (на всякий случай)**

In [0]:
import pickle
from lightfm import LightFM
from scipy.sparse import load_npz

ui_matrix = load_npz("inters.npz")
if_matrix = load_npz("ifeat.npz")
with open('4model.pickle', 'rb') as f:
  model = pickle.load(f)

**Просмотр получившихся эмбеддингов**

In [41]:
model.user_embeddings

array([[-0.06772878, -0.00335138, -0.10878259, ..., -0.02534712,
         0.30425188, -0.42803472],
       [ 0.01339718,  0.0072026 ,  0.11565433, ...,  0.05977984,
         0.0328368 , -0.29393238],
       [ 0.07081579,  0.15077028,  0.04871227, ..., -0.0527384 ,
        -0.18156216,  0.1331907 ],
       ...,
       [-0.3145021 , -0.06457923, -0.04856536, ..., -0.10562722,
         0.00377612, -0.14853296],
       [-0.34189966,  0.03382048,  0.00869182, ..., -0.17067395,
        -0.12272345,  0.07227996],
       [-0.12550026, -0.12913477, -0.04429622, ...,  0.04949018,
         0.2337197 ,  0.07340776]], dtype=float32)

In [42]:
model.item_embeddings

array([[ 0.00924585,  0.36714908, -0.27765334, ...,  0.19303343,
        -0.02979052,  0.0007598 ],
       [ 0.37882313, -0.21066907,  0.3415496 , ...,  0.00216993,
        -0.35179698, -0.47179192],
       [ 0.14987108,  0.25425395,  0.14173736, ...,  0.02535313,
        -0.4997711 , -0.22649872],
       ...,
       [ 0.02213195,  0.00182157,  0.00806212, ...,  0.00510067,
         0.05406705, -0.03515825],
       [-0.00280513, -0.00742661, -0.0023408 , ...,  0.00221562,
        -0.00711163, -0.02296642],
       [ 0.00821936,  0.00703982, -0.02469675, ..., -0.00162828,
         0.0556627 , -0.02865832]], dtype=float32)

# Предсказание и сабмит

**Ранжирование**

In [0]:
import pandas as pd
import numpy as np

In [0]:
random_benchmark = pd.read_csv('random_benchmark.csv')

In [45]:
print(random_benchmark.shape)
random_benchmark.head()

(3018186, 2)


,userId,itemId
0,1,242249
1,1,117266
2,1,123441
3,1,148575
4,1,155695


In [0]:
random_benchmark['scores'] = model.predict(np.array(random_benchmark.userId), np.array(random_benchmark.itemId), item_features=if_matrix, num_threads=4) 

In [0]:
random_benchmark = random_benchmark.sort_values(by=['userId', 'scores'], ascending=[True,False])

In [0]:
random_benchmark.drop('scores', axis=1, inplace=True)

In [49]:
print(random_benchmark.shape)
random_benchmark.head()

(3018186, 2)


,userId,itemId
68,1,106661
248,1,178123
438,1,201223
178,1,307942
459,1,260888


**Посылка с решением**

In [0]:
random_benchmark.to_csv("submission_lst_ver.csv", index=False)

In [0]:
!kaggle competitions submit -c recsys-iad-challenge -f submission_lst_ver.csv -m "Submission_Last_mb"

100% 35.7M/35.7M [00:08<00:00, 4.31MB/s]
Successfully submitted to Articles RecSys